In [1]:
import torch
import torch.nn as nn
import os,glob
import numpy as np
%pylab
import seaborn as sns
%matplotlib inline
from torch.utils.data import Dataset

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


In [8]:
save_path='/home/jaehun/workspace/incubator-tvm/result/GCN'
tasks = ('conv1d', 'conv1d_transpose',  'conv2d_transpose')


In [115]:
tasks = ('conv1d', 'conv1d_transpose',  'conv2d_transpose')
cost_path = list(map(lambda x: os.path.join(save_path,x, 'label.npy'), tasks))
feature_path = list(map(lambda x: os.path.join(save_path,x, 'batch_1.npy'), tasks))
_feature = []
for fea in feature_path:
    _feature.append(np.load(fea))
_cost = []
for cost in cost_path:
    _cost.append(np.load(cost))
np_feature=np.vstack(_feature)
np_cost=np.vstack(_cost)


In [116]:
n_task=int(np.ceil(len(np_feature)/100))
way=5
shot=10
ways = np.random.randint(0,n_task, way)

In [121]:
np_feature=np_feature.squeeze()

In [125]:
ways = np.random.randint(0, len(_cost), 5)
arr=np.take(np_feature, ways,axis=0)

In [127]:
arr

array([[-5.96768141e-01, -1.57110468e-01,  1.27489626e-01,
        -1.07097231e-01, -5.91746449e-01,  9.76049602e-01,
        -3.18904132e-01, -2.49113832e-02, -3.18211377e-01,
         1.96984828e-01, -6.13038763e-02, -2.38281876e-01,
        -3.67958367e-01,  2.24460452e-03, -4.47734892e-01,
        -6.09340668e-01,  4.63946164e-03, -3.38271141e-01,
        -6.33602515e-02, -5.26988983e-01, -1.03177875e-01,
        -1.96275994e-01,  1.60406327e+00, -2.62843966e-01,
        -1.95118099e-01,  1.88272655e-01, -1.97082758e-01,
        -5.04941523e-01, -1.61648318e-01,  8.01383108e-02,
        -1.26292229e-01, -2.82298237e-01, -3.73079628e-02,
        -2.40337849e-01, -4.35781389e-01, -2.63505429e-03,
         1.01374757e+00, -2.52691746e-01, -2.22227544e-01,
        -3.08710337e-01, -3.28129172e-01, -5.53236544e-01,
        -4.57081079e-01, -4.14371133e-01, -2.22368479e-01,
        -2.64764428e-01, -4.12255496e-01, -1.94456026e-01,
        -2.38458782e-01, -2.48274043e-01, -3.03335071e-0

In [30]:

candidate_cost = np.take(_cost, 5)
candidate_curve = np.take(_curve, 5)


In [32]:
cow

array([208,  72,  80, 157,  20])

In [130]:
class GraphDataset(Dataset):
    def __init__(self, ways=5, shot=5, test_shot=5, size=10000000, transform=None):
        self.shot = shot
        self.ways = ways
        self.test_shot = test_shot
        self.__save_path='/home/jaehun/workspace/incubator-tvm/result/GCN'
        self.transform=transform
        self.__tasks = ('conv1d', 'conv1d_transpose',  'conv2d_transpose')
        self.__cost_path = list(map(lambda x: os.path.join(self.__save_path,x, 'label.npy'), self.__tasks))
        self.__feature_path = list(map(lambda x: os.path.join(self.__save_path,x, 'batch_1.npy'), self.__tasks))
        _feature = []
        for fea in self.__feature_path:
            _feature.append(np.load(fea))
        _cost = []
        for cost in self.__cost_path:
            _cost.append(np.load(cost))
        self.__features=np.vstack(_feature).squeeze()
        self.__costs=np.vstack(_cost).squeeze()
        self.__n_task=int(np.ceil(len(self.__costs)/100))
        self.__cost_len=len(self.__costs)
    def __len__(self):
        return self.__cost_len * self.shot*self.ways
    def candidate(self,ways,shot):
        _list=[]
        for way in ways:
            if way==self.__n_task:
                _list.extend(random.sample(range(way*100,self.__cost_len), shot))
            else:
                _list.extend(random.sample(range(way*100,(way+1)*100), shot))
        return np.array(_list)
    def __getitem__(self, idx):
        batch = dict()
        ways = np.array(random.sample(range(self.__n_task), self.ways))
        
        candidate=self.candidate(ways,self.shot)
        train_data=np.take(self.__features, candidate,axis=0)
        train_label=np.take(self.__costs, candidate,axis=0)
        
        candidate=self.candidate(ways,self.test_shot)
        test_data=np.take(self.__features, candidate,axis=0)
        test_label=np.take(self.__costs, candidate,axis=0)

        train_data = torch.tensor(train_data, dtype=torch.float32)
        test_data = torch.tensor(test_data, dtype=torch.float32)
        batch['train'] = (train_data, train_label)
        batch['test'] = (test_data, test_label)
        return batch

In [132]:
t=GraphDataset()
dataset_loader = torch.utils.data.DataLoader(t,
                                             batch_size=2, shuffle=False,
                                             num_workers=1)
for data in dataset_loader:
    print(data['train'][0].shape)
    print(data['train'][1].shape)
    break



torch.Size([2, 25, 128])
torch.Size([2, 25, 2])


In [109]:
data['train'][0].shape

torch.Size([2, 25])

In [110]:
data['train'][1].shape

torch.Size([2, 25])

In [ ]:
_cost=[]
for cost in cost_path:
    _cost.extend(np.load(cost).tolist())
sns.distplot(_cost)

In [ ]:
_curve=[]
for curve in curve_path:
    _curve.append(np.load(curve)[:,:720])
_cost=[]
for cost in cost_path:
    np_cost=np.load(cost)
    scalar=MinMaxScaler()
    np_cost=scalar.fit_transform(np_cost.reshape(-1,1)).squeeze()
    _cost.append(np_cost)


In [42]:
from torchmeta.datasets.helpers import omniglot
from torchmeta.utils.data import BatchMetaDataLoader

dataset = omniglot("data", ways=4, shots=2, test_shots=15, meta_train=True, download=True)
dataloader = BatchMetaDataLoader(dataset, batch_size=1, num_workers=4)

for batch in dataloader:
    train_inputs, train_targets = batch["train"]
    print('Train inputs shape: {0}'.format(train_inputs.shape))    # (16, 25, 1, 28, 28)
    print('Train targets shape: {0}'.format(train_targets.shape))  # (16, 25)
    
    test_inputs, test_targets = batch["test"]
    print('Test inputs shape: {0}'.format(test_inputs.shape))      # (16, 75, 1, 28, 28)
    print('Test targets shape: {0}'.format(test_targets.shape))    # (16, 75)
    break

Train inputs shape: torch.Size([1, 8, 1, 28, 28])
Train targets shape: torch.Size([1, 8])
Test inputs shape: torch.Size([1, 60, 1, 28, 28])
Test targets shape: torch.Size([1, 60])


In [37]:
train_targets

tensor([[2, 2, 3, 3, 1, 1, 0, 0],
        [0, 0, 3, 3, 2, 2, 1, 1],
        [2, 2, 1, 1, 0, 0, 3, 3],
        [2, 2, 0, 0, 3, 3, 1, 1],
        [0, 0, 3, 3, 2, 2, 1, 1],
        [0, 0, 1, 1, 2, 2, 3, 3],
        [0, 0, 3, 3, 1, 1, 2, 2],
        [0, 0, 1, 1, 2, 2, 3, 3],
        [0, 0, 2, 2, 1, 1, 3, 3],
        [0, 0, 1, 1, 3, 3, 2, 2],
        [0, 0, 1, 1, 3, 3, 2, 2],
        [0, 0, 3, 3, 2, 2, 1, 1],
        [3, 3, 0, 0, 2, 2, 1, 1],
        [3, 3, 1, 1, 2, 2, 0, 0],
        [0, 0, 1, 1, 2, 2, 3, 3],
        [1, 1, 0, 0, 3, 3, 2, 2]])